In [1]:
# Copy this file and then run an experiment
# Import modules
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import importlib
import pickle  # Use this to save results so we don't need to rerun experiments
rf = importlib.import_module("random-forests")
misclass_rate = rf.random_forests.misclassification_rate
cmap = {"iris": "blue", "wine": "purple", "heart": "red", "titanic": "green", "mushroom": "orange"}
def comp_mean_sd(arr):
    return np.array([np.mean(arr), np.std(arr)])
def m_sd(results):
    ret = {}
    for k in results.keys():
        ret[k] = np.apply_along_axis(comp_mean_sd, 1, results[k])
    return ret

In [2]:
# Load all datasets to use
mushroom = np.loadtxt("agaricus-lepiota.data", dtype=str, delimiter=",")
wine = np.loadtxt("wine.data", delimiter=",")
iris = np.loadtxt("iris_csv.csv", dtype=str, delimiter=",")
heart = np.loadtxt("processed.cleveland.data", dtype=str, delimiter=",")
titanic = pd.read_csv("titanic.csv").drop(columns=["PassengerId", "Name", "Ticket", "Cabin"]).dropna()

In [3]:
# Clean data
def str_in_arr(arr, str):
    return not np.isin(str, arr)
barr_mushroom = np.apply_along_axis(str_in_arr, 1, mushroom, "?")
mushroom = mushroom[barr_mushroom, :]
barr_heart = np.apply_along_axis(str_in_arr, 1, heart, "?")
heart = heart[barr_heart, :]
heart = heart.astype(float)

In [4]:
# Separate into data and labels
iris_data = iris[1:, :4].astype(float)
iris_labels = iris[1:, 4]
wine_data = wine[:, 1:]
wine_labels = wine[:, 0].astype(int)
heart_data = heart[:, :-1]
heart_labels = heart[:, -1].astype(int)
# Turn heart_labels into array of 0, 1.
# 1 indicates presence of any type of heart disease, 0 indicates no presence
heart_labels_barr = heart_labels >= 1
heart_labels[heart_labels_barr] = 1
mushroom_data = mushroom[:, 1:]
mushroom_labels = mushroom[:, 0]
titanic_data = titanic.loc[:, titanic.columns!="Survived"].to_numpy()  # feature_type: complex
titanic_labels = titanic["Survived"].to_numpy()

In [5]:
# Record the feature types
iris_ftype = "continuous"
heart_ftype = np.array([0, 1, 1, 0, 0, 1, 1, 0, 1, 0, 1, 0, 1])
wine_ftype = "continuous"
mushroom_ftype = "categorical"
titanic_ftype = np.array([1, 1, 0, 0, 0, 0, 1])

In [6]:
# Set the seed so test train split it always the same for all tests
np.random.seed(1)
datasets = {"iris": (*train_test_split(iris_data, iris_labels, stratify=iris_labels), "continuous"),
            "wine": (*train_test_split(wine_data, wine_labels, stratify=wine_labels), "continuous"),
            "heart": (*train_test_split(heart_data, heart_labels, stratify=heart_labels), heart_ftype),
            "mushroom": (*train_test_split(mushroom_data, mushroom_labels, stratify=mushroom_labels), "categorical"),
            "titanic": (*train_test_split(titanic_data, titanic_labels, stratify=titanic_labels), titanic_ftype)}

In [31]:
# number of experiments to run for each parameter choice
nexp = 10
results = {}
ntrees=np.arange(1,11)

In [40]:
dataset = "iris"
X_train, X_test, y_train, y_test, ftype = datasets[dataset]
result = np.empty((3, len(ntrees), nexp), dtype=float)
m_feats = np.array([1, int(np.floor(np.sqrt(X_train.shape[1]))), X_train.shape[1]])
n_cands = 3*m_feats
np.random.seed(1)
for i, m_feat in enumerate(m_feats):
    n_cand = n_cands[i]
    for j, ntree in enumerate(ntrees):
        for k in range(nexp):
            random_forest = rf.RandomForest(n_trees=ntree, n_candidates=n_cand, max_depth=10)
            random_forest.fit(X_train, y_train, feature_type=ftype, m_features=m_feat)
            result[i, j, k] = misclass_rate(random_forest.predict(X_test), y_test)
results[dataset] = result

In [43]:
dataset = "wine"
X_train, X_test, y_train, y_test, ftype = datasets[dataset]
result = np.empty((3, len(ntrees), nexp), dtype=float)
m_feats = np.array([1, int(np.floor(np.sqrt(X_train.shape[1]))), X_train.shape[1]])
n_cands = 3*m_feats
np.random.seed(1)
for i, m_feat in enumerate(m_feats):
    n_cand = n_cands[i]
    for j, ntree in enumerate(ntrees):
        for k in range(nexp):
            random_forest = rf.RandomForest(n_trees=ntree, n_candidates=n_cand, max_depth=10)
            random_forest.fit(X_train, y_train, feature_type=ftype, m_features=m_feat)
            result[i, j, k] = misclass_rate(random_forest.predict(X_test), y_test)
results[dataset] = result

In [44]:
dataset = "heart"
X_train, X_test, y_train, y_test, ftype = datasets[dataset]
result = np.empty((3, len(ntrees), nexp), dtype=float)
m_feats = np.array([1, int(np.floor(np.sqrt(X_train.shape[1]))), X_train.shape[1]])
n_cands = 3*m_feats
np.random.seed(1)
for i, m_feat in enumerate(m_feats):
    n_cand = n_cands[i]
    for j, ntree in enumerate(ntrees):
        for k in range(nexp):
            random_forest = rf.RandomForest(n_trees=ntree, n_candidates=n_cand, max_depth=10)
            random_forest.fit(X_train, y_train, feature_type=ftype, m_features=m_feat)
            result[i, j, k] = misclass_rate(random_forest.predict(X_test), y_test)
results[dataset] = result

In [45]:
dataset = "titanic"
X_train, X_test, y_train, y_test, ftype = datasets[dataset]
result = np.empty((3, len(ntrees), nexp), dtype=float)
m_feats = np.array([1, int(np.floor(np.sqrt(X_train.shape[1]))), X_train.shape[1]])
n_cands = 3*m_feats
np.random.seed(1)
for i, m_feat in enumerate(m_feats):
    n_cand = n_cands[i]
    for j, ntree in enumerate(ntrees):
        for k in range(nexp):
            random_forest = rf.RandomForest(n_trees=ntree, n_candidates=n_cand, max_depth=10)
            random_forest.fit(X_train, y_train, feature_type=ftype, m_features=m_feat)
            result[i, j, k] = misclass_rate(random_forest.predict(X_test), y_test)
results[dataset] = result

In [46]:
dataset = "mushroom"
X_train, X_test, y_train, y_test, ftype = datasets[dataset]
result = np.empty((3, len(ntrees), nexp), dtype=float)
m_feats = np.array([1, int(np.floor(np.sqrt(X_train.shape[1]))), X_train.shape[1]])
n_cands = 3*m_feats
np.random.seed(1)
for i, m_feat in enumerate(m_feats):
    n_cand = n_cands[i]
    for j, ntree in enumerate(ntrees):
        for k in range(nexp):
            print(i, j, k)
            random_forest = rf.RandomForest(n_trees=ntree, n_candidates=n_cand, max_depth=10)
            random_forest.fit(X_train, y_train, feature_type=ftype, m_features=m_feat)
            result[i, j, k] = misclass_rate(random_forest.predict(X_test), y_test)
results[dataset] = result

0 0 0
0 0 1
0 0 2
0 0 3
0 0 4
0 0 5
0 0 6
0 0 7
0 0 8
0 0 9
0 1 0
0 1 1
0 1 2
0 1 3
0 1 4
0 1 5
0 1 6
0 1 7
0 1 8
0 1 9
0 2 0
0 2 1
0 2 2
0 2 3
0 2 4
0 2 5
0 2 6
0 2 7
0 2 8
0 2 9
0 3 0
0 3 1
0 3 2
0 3 3
0 3 4
0 3 5
0 3 6
0 3 7
0 3 8
0 3 9
0 4 0
0 4 1
0 4 2
0 4 3
0 4 4
0 4 5
0 4 6
0 4 7
0 4 8
0 4 9
0 5 0
0 5 1
0 5 2
0 5 3
0 5 4
0 5 5
0 5 6
0 5 7
0 5 8
0 5 9
0 6 0
0 6 1
0 6 2
0 6 3
0 6 4
0 6 5
0 6 6
0 6 7
0 6 8
0 6 9
0 7 0
0 7 1
0 7 2
0 7 3
0 7 4
0 7 5
0 7 6
0 7 7
0 7 8
0 7 9
0 8 0
0 8 1
0 8 2
0 8 3
0 8 4
0 8 5
0 8 6
0 8 7
0 8 8
0 8 9
0 9 0
0 9 1
0 9 2
0 9 3
0 9 4
0 9 5
0 9 6
0 9 7
0 9 8
0 9 9
1 0 0
1 0 1
1 0 2
1 0 3
1 0 4
1 0 5
1 0 6
1 0 7
1 0 8
1 0 9
1 1 0
1 1 1
1 1 2
1 1 3
1 1 4
1 1 5
1 1 6
1 1 7
1 1 8
1 1 9
1 2 0
1 2 1
1 2 2
1 2 3
1 2 4
1 2 5
1 2 6
1 2 7
1 2 8
1 2 9
1 3 0
1 3 1
1 3 2
1 3 3
1 3 4
1 3 5
1 3 6
1 3 7
1 3 8
1 3 9
1 4 0
1 4 1
1 4 2
1 4 3
1 4 4
1 4 5
1 4 6
1 4 7
1 4 8
1 4 9
1 5 0
1 5 1
1 5 2
1 5 3
1 5 4
1 5 5
1 5 6
1 5 7
1 5 8
1 5 9
1 6 0
1 6 1
1 6 2
1 6 3
1 6 4
1 6 5
1 6 

In [24]:
results

{'iris': array([[0.02631579, 0.02631579, 0.02631579, 0.02631579, 0.07894737,
         0.05263158, 0.02631579, 0.02631579, 0.05263158, 0.02631579],
        [0.02631579, 0.05263158, 0.02631579, 0.02631579, 0.05263158,
         0.02631579, 0.02631579, 0.02631579, 0.02631579, 0.02631579],
        [0.07894737, 0.05263158, 0.05263158, 0.02631579, 0.05263158,
         0.02631579, 0.02631579, 0.02631579, 0.02631579, 0.02631579]]),
 'wine': array([[0.02222222, 0.11111111, 0.04444444, 0.06666667, 0.04444444,
         0.02222222, 0.02222222, 0.11111111, 0.04444444, 0.11111111],
        [0.06666667, 0.04444444, 0.08888889, 0.08888889, 0.04444444,
         0.06666667, 0.06666667, 0.06666667, 0.08888889, 0.04444444],
        [0.02222222, 0.06666667, 0.11111111, 0.06666667, 0.06666667,
         0.06666667, 0.04444444, 0.08888889, 0.11111111, 0.08888889]]),
 'heart': array([[0.21333333, 0.26666667, 0.21333333, 0.22666667, 0.29333333,
         0.16      , 0.26666667, 0.29333333, 0.24      , 0.25333333]

In [25]:
po = open("vary_m_features.pickle", "wb")
pickle.dump(results, po)
po.close()

In [26]:
mean_sd = m_sd(results)

In [28]:
arr = np.zeros((3,4,5))

In [29]:
arr[1, 2, 3] = 1

In [30]:
arr

array([[[0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.]],

       [[0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 1., 0.],
        [0., 0., 0., 0., 0.]],

       [[0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.]]])

In [47]:
results.keys()

dict_keys(['iris', 'wine', 'heart', 'titanic', 'mushroom'])

In [48]:
results['mushroom']

array([[[0.00070872, 0.02126152, 0.01275691, 0.00354359, 0.00850461,
         0.00708717, 0.02622254, 0.01771793, 0.03260099, 0.01842665],
        [0.04039688, 0.00850461, 0.02197023, 0.00708717, 0.01771793,
         0.02197023, 0.04961021, 0.02126152, 0.0205528 , 0.05598866],
        [0.00354359, 0.00354359, 0.00566974, 0.00212615, 0.        ,
         0.01771793, 0.00496102, 0.01913536, 0.02409639, 0.        ],
        [0.00850461, 0.00779589, 0.01984408, 0.00141743, 0.00496102,
         0.00708717, 0.00141743, 0.01133948, 0.01488306, 0.0042523 ],
        [0.00141743, 0.00283487, 0.00141743, 0.00070872, 0.        ,
         0.00070872, 0.        , 0.00070872, 0.00212615, 0.        ],
        [0.        , 0.        , 0.0042523 , 0.        , 0.        ,
         0.00070872, 0.00212615, 0.00070872, 0.        , 0.00070872],
        [0.00212615, 0.        , 0.0042523 , 0.        , 0.        ,
         0.        , 0.00566974, 0.        , 0.00070872, 0.        ],
        [0.        , 0.0014

In [49]:
po = open("vary_m_features_n_trees.pickle", "wb")
pickle.dump(results, po)
po.close()